# Experimento 1.Parte 2.Análisis de la arquitectura usada en "A Preliminary Study on Deep Transfer Learning Applied to Image Classification for Small Datasets"

En este experimento comprobaremos como afecta la implementación de capas de dropout, batchnormalization y funciones de activación ReLU sobre la red propuesta en "A Preliminary Study on Deep Transfer Learning Applied to Image Classification for Small Datasets" con los hiperparámetros elegidos en el Experimento 1.Parte 2.

## Librerías usadas.

In [1]:
import tensorflow as tf
gpus= tf.config.experimental.list_physical_devices('GPU')
print(gpus)
tf.config.experimental.set_memory_growth(gpus[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import math 
from glob import glob
from matplotlib import pyplot as plt
import os
from tqdm import tqdm
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample

## Definición de rutas y variables globales

In [3]:
#Rutas de los datos.
 
data_dir = os.path.dirname(os.path.realpath("../TFG/Datos/HAM10000_metadata.csv"))



csv_path = os.path.realpath(data_dir + "/HAM10000_metadata.csv")

#Variables globales

altura = 50
longitud = 50
clases = 7


print(data_dir)

print(csv_path)



/home/antgarnie/Escritorio/TFG/Datos
/home/antgarnie/Escritorio/TFG/Datos/HAM10000_metadata.csv


## Creación del marco de datos.

In [4]:
#Inicializando el dataFrame

dataFrame=pd.read_csv(csv_path)

#Mezclando carpetas.

all_image_path = glob(os.path.join(data_dir, '*', '*'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}

# Inicializando diccionario de categorías

lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

#Añadiendo columnas al dataFrame para que sea más legible.

dataFrame['path'] = dataFrame['image_id'].map(imageid_path_dict.get)
dataFrame['cell_type'] = dataFrame['dx'].map(lesion_type_dict.get) 
dataFrame['cell_type_idx'] = pd.Categorical(dataFrame['cell_type']).codes
dataFrame.head()


,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,/home/antgarnie/Escritorio/TFG/Datos/HAM10000_...,Benign keratosis,2


## Preparación de la red



In [5]:
def full_build_cnn_soco(withBatchNormalization = False, withDropout=False):
    
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(32, (3,3),(1,1),input_shape=(altura,longitud,3)))
    model.add(tf.keras.layers.PReLU())
    if(withBatchNormalization):
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Conv2D(32, (3,3),(1,1)))
    model.add(tf.keras.layers.PReLU())
    if(withBatchNormalization):
        model.add(tf.keras.layers.BatchNormalization())
    
    model.add(tf.keras.layers.MaxPooling2D((2,2)))

 
    model.add(tf.keras.layers.Conv2D(64, (3,3)))
    model.add(tf.keras.layers.PReLU())
    if(withBatchNormalization):
        model.add(tf.keras.layers.BatchNormalization())
        
    model.add(tf.keras.layers.MaxPooling2D((2,2)))
    
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(128))
    model.add(tf.keras.layers.PReLU())
    if(withDropout):
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(clases,activation='softmax'))

    return model

## Se procede a crear un método que permita balancear la carga de imágenes

In [6]:
def balanced_dataset(df):
    df_balanced = pd.DataFrame()
    #df = pd.DataFrame()
    
    for cat in df['cell_type_idx'].unique():
        temp = resample(df[df['cell_type_idx'] == cat], 
                        replace=True,     # sample with replacement
                        n_samples=2500,   # to match majority class
                        random_state=123) # reproducible results

        # Combine majority class with upsampled minority class
        df_balanced = pd.concat([df_balanced, temp])
 
    df_balanced['cell_type'].value_counts()

    return df_balanced

def load_img_data(size, df, balanced=False):
    """
        ..
        first we should normalize the image from 0-255 to 0-1
    """
    
    img_h, img_w = size, size
    imgs = []
    
    if balanced:
        df = balanced_dataset(df)
    
    image_paths = list(df['path'])

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        img = img.astype(np.float32) / 255.
        #img = np.asarray(Image.open(image_paths[i]).resize((size,size)))
        imgs.append(img)

    imgs = np.stack(imgs, axis=0)
    print(imgs.shape)

    #imgs = imgs.astype(np.float32) / 255.
    
    return imgs, df['cell_type_idx'].values

## Cargamos los datos y creamos los casos a experimentar.

In [7]:
def load_general_data():
    
    imgs, target = load_img_data(altura, dataFrame, balanced=True)
    
    x_train, x_transferLearning, y_train, y_transferLearning = train_test_split(imgs, target, test_size=0.60)
       
    source_data = [ x_transferLearning , y_transferLearning ]
    target_data = [ x_train , y_train ]
    
    x_train,x_test,y_train,y_test = train_test_split(target_data[0], target_data[1], test_size=0.70)
    
    train_data = [x_train,y_train]
    test_data = [x_test,y_test]
    
    return source_data,train_data,test_data


def get_data_for_ex(source_data,train_data,test_data):
    
    x_train = source_data[0]
    y_train = source_data[1]
    
    x_retrain = train_data[0]
    y_retrain = train_data[1]
    
    percent = math.floor(len(test_data[0])/100*30)
       
    x_validation = test_data[0][0:percent]
    y_validation = test_data[1][0:percent]
    
    
    x_test = test_data[0][percent:-1]
    y_test = test_data[1][percent:-1]
    
    return x_train,x_retrain,x_test,x_validation,y_train,y_retrain,y_test,y_validation


#####################################
# Definimos los parámetros globales #
#####################################

ITERATIONS_PER_EXP = 10
BATCH_SIZE = 16
EPOCHS = 15
LEARNING_RATE=0.0001

In [8]:
source_data,train_data,test_data = load_general_data()
x_train,x_retrain,x_test,x_validation,y_train,y_retrain,y_test,y_validation = get_data_for_ex(source_data,train_data,test_data)

100%|██████████| 17500/17500 [01:58<00:00, 148.27it/s]


(17500, 50, 50, 3)


In [ ]:
#Experimento 1. Red soco con Dropout.
res11,res12,evaluations,evaluations1 = run_experiment(0,20)

#Experimento 2. Red soco con BN y Dropout.
res21,res22,evaluations,evaluations2 = run_experiment(1,15)

#Experimento 3. Red soco con BN y sin Dropout.
res31,res32,evaluations,evaluations3 = run_experiment(2,15)

In [ ]:
def run_train_w_model(model,epochs,output_layer):
        
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001,amsgrad=True)
        
    earlyStopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                patience = 3,
                                                mode = 'min')    
    
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    history = model.fit(x_train, y_train,validation_data=(x_validation, y_validation),epochs=epochs,
                        callbacks=[earlyStopping],batch_size = BATCH_SIZE)
    
    print(len(x_test))
    
    evaluation = model.evaluate(x_test, y_test)
        
    return history,evaluation,model
    
def re_train(model,checkpoint,epochs):
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001,amsgrad=True)

    earlyStopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
                                                patience = 3,
                                                mode = 'min')    
    
    cpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint,
                                                monitor="val_loss",
                                                mode="min",
                                                save_best_only=True,
                                                verbose=0)
    
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    history = model.fit(x_train, y_train,validation_data=(x_validation, y_validation),epochs=epochs,callbacks=[earlyStopping,cpoint],batch_size = BATCH_SIZE)
    evaluation = model.evaluate(x_test, y_test)
    return history,evaluation,model

In [ ]:
def run_experiment(index_model,epoch,iterations = ITERATIONS_PER_EXP):
    result = []
    result_post_tf = []
    evaluations = []
    evaluations_post_tf = []
    dense = False
    
    for i in range(iterations):
        checkpoint ="../TFG/Modelos/balanced_model_functionalv2.h5"
        
        if index_model == 0:
            model = full_build_cnn_soco(withBatchNormalization = False, withDropout=True)
        if index_model == 1:
            model = full_build_cnn_soco(withBatchNormalization = True, withDropout=True)
        if index_model == 2:
            model = full_build_cnn_soco(withBatchNormalization = True, withDropout=False)
        
        h,e,tf_model = run_train_w_model(model,epoch,dense)
        
        result.append(h)
        evaluations.append(e)

        layers = tf_model.layers[0:-1]
        for layer in layers:
            layer.trainable = False
            
        h_retrain,e_retrain,model = re_train(tf_model,checkpoint,epoch)

        result_post_tf.append(h_retrain)
        evaluations_post_tf.append(e_retrain)
        
        
        
        print("########################################################")
        print("Iteración "+str(i+1) +" de "+ str(iterations))
        print("########################################################")
        
    return result,result_post_tf,evaluations,evaluations_post_tf

## Métodos de representación gráfica

In [ ]:
def plot_acc_for_ex1_w_test_score(res,evaluations,epochs,name):
    plt.figure(figsize=(12,8))
    
    for i in range(len(res)):
        precisiones=[]
        precisiones.append(0.0)
        for e in res[i].history["accuracy"]:
            precisiones.append(e)
            
        plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1], label="train_acc")
        plt.plot(5,evaluations[i][1],'bo')
        
    plt.title("Training Loss and Accuracy - {}".format(name))
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()

In [ ]:
def plot_acc_for_ex1(res,epochs,name):
    plt.figure(figsize=(12,8))
    
    for i in range(len(res)):
        precisiones=[]
        precisiones.append(0.0)
        for e in res[i].history["accuracy"]:
            precisiones.append(e)
            
        plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1], label="train_acc")
    plt.title("Training Loss and Accuracy - {}".format(name))
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.show()

In [ ]:
def plot_acc_for_experiments(res,res1,evaluations,evaluations1,epochs,name):
    plt.figure(figsize=(12,8))
    
    for i in range(len(res)):
        precisiones=[]
        precisiones.append(0.0)
        for e in res[i].history["accuracy"]:
            precisiones.append(e)
        
        precisiones1=[]
        precisiones1.append(0.0)
        for e in res1[i].history["accuracy"]:
            precisiones1.append(e)
        
        if i == 0:
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1], label="train_acc",color='green')
            plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1], label="train_acc",color='blue')
        else:
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1],color='green')
            plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1],color='blue')
            plt.plot(5,evaluations[i][1],'bo',color='green')
            plt.plot(5,evaluations1[i][1],'bo',color='blue')
        
    plt.title("Training Loss and Accuracy - {}".format(name))
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.savefig("Exp_1_"+"Training "+str(value)+" and validation "+str(value)+".jpg")
    plt.legend()
    plt.show()

In [ ]:
def plot_trainAcc_vs_valAcc_for_experiments(res,res1,epochs,name,value="loss"):
    plt.figure(figsize=(12,8))
    
    for i in range(len(range(5))):
        precisiones=[]
        precisiones.append(0.0)
        for e in res[i].history[value]:
            precisiones.append(e)
            
        #precisiones.append(0.0)
        for e in res1[i].history[value]:
            precisiones.append(e)
        
        precisiones1=[]
        precisiones1.append(0.0)
        for e in res[i].history["val_"+str(value)]:
            precisiones1.append(e)

        #precisiones1.append(0.0)
        for e in res1[i].history["val_"+str(value)]:
            precisiones1.append(e)
        
        if i == 0:
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1], label="train_"+str(value),color='green')
            plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1], label="val_"+str(value),color='blue')
        else:
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1],color='green')
            plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1],color='blue')
        
    plt.title("Training "+str(value)+" and validation "+str(value))
    plt.xlabel("Epoch")
    plt.ylabel(str(value).capitalize())
    plt.legend()
    plt.savefig("Exp_1_BN_"+"Training "+str(value)+" and validation "+str(value)+".jpg")
    plt.show()

In [ ]:
plot_trainAcc_vs_valAcc_for_experiments(res31,res32,17,"","accuracy")

## Representamos gráficamente los resultados obtenidos al experimentar con el tamaño del lote

In [ ]:
def compute_values(resx1,resx2,i,val):
    res=[]
    if val != "val_loss":
        res.append(0.0)
    for e in resx1[i].history[val]:
        res.append(e)
    for e in resx2[i].history[val]:
        res.append(e)
    return res




def plot_acc_all_experiments(res11,res12,res21,res22,res31,res32,epochs):
    plt.figure(figsize=(16,10))
    
    acc = "val_accuracy"
    
    for i in range(10):
        
        precisiones = compute_values(res11,res12,i,acc)
            
        precisiones1 = compute_values(res21,res22,i,acc)
        
        precisiones2 = compute_values(res31,res32,i,acc)
        
        
        loss = compute_values(res11,res12,i,"val_loss")
        
        loss1 = compute_values(res21,res22,i,"val_loss")
        
        loss2 = compute_values(res31,res32,i,"val_loss")
        

        if i == 0:
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1], label="Dropout",color='green')
            #plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1], label="BachNormalization y Dropout",color='blue')
            plt.plot(np.arange(0, epochs+1), precisiones2[0:epochs+1], label="BachNormalization sin Dropout",color='red')
            #plt.plot(np.arange(0, epochs+1), precisiones3[0:epochs+1], label="Amsgrad",color='aqua')
            #plt.plot(np.arange(0, epochs+1), precisiones4[0:epochs+1], label="Adamax",color='red')
            #plt.plot(np.arange(0, epochs+1), precisiones5[0:epochs+1], label="Nadam",color='deeppink')
            #plt.plot(np.arange(0, epochs+1), precisiones6[0:epochs+1], label="Adadelta",color='yellowgreen')
            
            plt.plot(np.arange(0, epochs+1), loss[0:epochs+1],color='green')
            #plt.plot(np.arange(0, epochs+1), loss1[0:epochs+1],color='blue')
            plt.plot(np.arange(0, epochs+1), loss2[0:epochs+1],color='red')
            #plt.plot(np.arange(0, epochs+1), loss3[0:epochs+1],color='aqua')
            #plt.plot(np.arange(0, epochs+1), loss4[0:epochs+1], label="Adamax",color='red')
            #plt.plot(np.arange(0, epochs+1), loss5[0:epochs+1], label="Nadam",color='deeppink')
            #plt.plot(np.arange(0, epochs+1), loss6[0:epochs+1], label="Adadelta",color='yellowgreen')
            
        else:
            
            plt.plot(np.arange(0, epochs+1), precisiones[0:epochs+1],color='green')
            #plt.plot(np.arange(0, epochs+1), precisiones1[0:epochs+1],color='blue')
            plt.plot(np.arange(0, epochs+1), precisiones2[0:epochs+1],color='red')
            #plt.plot(np.arange(0, epochs+1), precisiones3[0:epochs+1],color='aqua')
            #plt.plot(np.arange(0, epochs+1), precisiones4[0:epochs+1], label="Adamax",color='red')
            #plt.plot(np.arange(0, epochs+1), precisiones5[0:epochs+1], label="Nadam",color='deeppink')
            #plt.plot(np.arange(0, epochs+1), precisiones6[0:epochs+1], label="Adadelta",color='yellowgreen')
            
            plt.plot(np.arange(0, epochs+1), loss[0:epochs+1],color='green')
            #plt.plot(np.arange(0, epochs+1), loss1[0:epochs+1],color='blue')
            plt.plot(np.arange(0, epochs+1), loss2[0:epochs+1],color='red')
            #plt.plot(np.arange(0, epochs+1), loss3[0:epochs+1],color='aqua')
            #plt.plot(np.arange(0, epochs+1), loss4[0:epochs+1],color='red')
            #plt.plot(np.arange(0, epochs+1), loss5[0:epochs+1],color='deeppink')
            #plt.plot(np.arange(0, epochs+1), loss6[0:epochs+1],color='yellowgreen')
            
        del precisiones
        del precisiones1
        del precisiones2


        
    plt.title("Training Accuracy and Test Results")
    plt.xlabel("Epoch")
    plt.ylabel("Loss/Accuracy")
    plt.legend()
    plt.savefig("Exp_1_2_kinds.jpg")
    plt.show()

In [ ]:
plot_acc_all_experiments(res11,res12,res21,res22,res31,res32,16)

## Creamos marcos de datos para analizar los resultados de evaluar los modelos 

In [ ]:
precision1 = []
precision2 = []
precision3 = []
precision4 = []
precision5 = []


for e in evaluations1:
    element = e[1]
    precision1.append(element)
    
for e in evaluations2:
    element = e[1]
    precision2.append(element)
    
for e in evaluations3:
    element = e[1]
    precision3.append(element)

    
d = {'Dropout': precision1,'BachNormalization y Dropout': precision2,'BachNormalization sin Dropout':precision3}
df = pd.DataFrame(data=d)

print(df)
print(df.mean())